In [65]:
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor 
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, GridSearchCV

In [67]:
df = pd.read_excel('c:/bc/base_line_hyeryeon_bomi.xlsx', header=0, sheet_name='original_data')

In [68]:
df['p_group']=pd.Categorical(df['p_group']).codes

In [69]:
del df['p_name']
del df['exp_min']
del df['YEAR_DAY']

In [72]:
del df['DAY_NAME']
del df['no']
del df['brand_name']

In [76]:
del df['total_price']
del df['broadcast']
del df['Unnamed: 0']

In [75]:
Y = np.array(df['total_price'])
Y

array([  2099000,   4371000,   3262000, ...,  50929000, 104392000,
        46608000], dtype=int64)

In [78]:
X = df

In [80]:
X

,m_code,p_code,p_group,unit_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,...,code,day_of_week_and_hour,52_week,holiday,brand_power,mask,parcel,rain,humidity,sunshine
0,100346,201072,6,39900,53,2019,1,1,6,0,...,0,30,1,1,16.386845,0.04095,0.52582,0.0,60.0,0.0
1,100346,201079,6,39900,110,2019,1,1,6,0,...,0,30,1,1,16.386845,0.04095,0.52582,0.0,60.0,0.0
2,100346,201072,6,39900,82,2019,1,1,6,20,...,0,30,1,1,16.386845,0.04095,0.52582,0.0,60.0,0.0
3,100346,201079,6,39900,174,2019,1,1,6,20,...,0,30,1,1,16.386845,0.04095,0.52582,0.0,60.0,0.0
4,100346,201072,6,39900,167,2019,1,1,6,40,...,0,30,1,1,16.386845,0.04095,0.52582,0.0,60.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35374,100448,201390,9,168000,91,2019,12,31,23,40,...,6923,47,53,0,13.968137,0.03906,1.28643,0.0,45.0,0.0
35375,100448,201384,9,158000,87,2020,1,1,0,0,...,6924,48,53,0,13.968137,0.03906,1.28643,0.0,66.0,0.0
35376,100448,201383,9,178000,286,2020,1,1,0,0,...,6924,48,53,0,13.968137,0.03906,1.28643,0.0,66.0,0.0
35377,100448,201390,9,168000,621,2020,1,1,0,0,...,6924,48,53,0,13.968137,0.03906,1.28643,0.0,66.0,0.0


In [81]:
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.2)

In [83]:
xgb = XGBRegressor(random_state=44)

xgb_param = {
    'min_child_weight': [1, 5, 10],
    'gamma': [0.5, 1, 1.5, 2, 5],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0],
    'max_depth': [3, 4, 5]
}

grid_xgb = GridSearchCV(xgb, param_grid=xgb_param)
grid_xgb.fit(trainX, trainY)

GridSearchCV(estimator=XGBRegressor(base_score=None, booster=None,
                                    colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, gamma=None,
                                    gpu_id=None, importance_type='gain',
                                    interaction_constraints=None,
                                    learning_rate=None, max_delta_step=None,
                                    max_depth=None, min_child_weight=None,
                                    missing=nan, monotone_constraints=None,
                                    n_estimators=100, n_jobs=None,
                                    num_parallel_tree=None, random_state=44,
                                    reg_alpha=None, reg_lambda=None,
                                    scale_pos_weight=None, subsample=None,
                                    tree_method=None, validate_parameters=None,
        

In [85]:
print('xgboost best param : ',grid_xgb.best_params_)
print('xgboost best accuracy : ',grid_xgb.best_score_)

#xgboost best param :  {'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}
#xgboost best accuracy :  0.9932464443644917

xgboost best param :  {'colsample_bytree': 1.0, 'gamma': 0.5, 'max_depth': 5, 'min_child_weight': 1, 'subsample': 0.8}
xgboost best accuracy :  0.9933622780465926


In [86]:
xgb_pred = grid_xgb.best_estimator_.predict(testX)

In [87]:
grid_xgb.score(testX, testY)

0.9963434653943949

In [91]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [92]:
mean_absolute_percentage_error(np.array(testY), xgb_pred)

4.671971903810377